# Autograd vs Bmad Jacobians
In this example, we track single particles through different elements and calculate the Jacobian matrix by using PyTorch autograd. We compare the results with bmad_standard, which uses analytic formulas to calculate the Jacobian. We also test functionalities such as element offsets and tilts. As of now, only transverse transformations are supported.

In [1]:
import torch
#from pytao import Tao
#import pytao
import time
import os
torch.set_printoptions(precision= 15, sci_mode=True)
#torch.__version__, pytao.__version__

In [2]:
from pathlib import Path

def find_repo(path):
    "Find repository root from the path's parents"
    for path in Path(path).parents:
        # Check whether "path/.git" exists and is a directory
        git_dir = path / ".git"
        if git_dir.is_dir():
            return path

import os
nb_dir = os.getcwd()
# Find the repo root where the script is
repo_path = str(find_repo(nb_dir))
repo_path

'/Users/jpga/Repositories/Bmad-X'

# Constants

# Drift tests

In [3]:
# Create drift
from bmadx import Drift

d = Drift(L=1.0)
d

Drift(L=1.0)

## Drift one particle test

In [4]:
# create incoming particle
from bmadx import Particle, M_ELECTRON

coords_t = torch.tensor([2e-3,3e-3,-3e-3,-1e-3,2e-3,-2e-3])
p_in = Particle(*coords_t,
                s=torch.tensor(0.0),
                p0c=torch.tensor(4.0e7),
                mc2 = M_ELECTRON)
p_in

Particle(x=tensor(2.000000094994903e-03), px=tensor(3.000000026077032e-03), y=tensor(-3.000000026077032e-03), py=tensor(-1.000000047497451e-03), z=tensor(2.000000094994903e-03), pz=tensor(-2.000000094994903e-03), s=tensor(0.), p0c=tensor(4.000000000000000e+07), mc2=510998.94999999995)

In [5]:
# Outgoing particle:
from bmadx import track_element

p_out = track_element(p_in, d)

# coordinates as tensor:
x_py = torch.hstack(p_out[:6])
x_py

tensor([5.006027407944202e-03, 3.000000026077032e-03, -4.002009052783251e-03,
        -1.000000047497451e-03, 1.994652673602104e-03, -2.000000094994903e-03])

In [6]:
# bmad lattice for comparison
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_drift.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out=tao.orbit_at_s(ele=1)

NameError: name 'Tao' is not defined

In [ ]:
# bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']])
x_tao

In [ ]:
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

## Drift Jacobian test

In [ ]:
# autodiff Jacobian
from torch.autograd.functional import jacobian

f_drift = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), d)[:6]

diff_coords = coords_t.clone().detach().requires_grad_(True)

J = jacobian(f_drift, diff_coords)
mat_py = torch.vstack(J)
mat_py

In [ ]:
# Bmad analytical Jacobian
drift_tao = tao.matrix(0,1)
mat_tao = torch.tensor(drift_tao['mat6'])
mat_tao

In [ ]:
# is it close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-14)

# Quadrupole tests

In [ ]:
# Create quad
from bmadx import Quadrupole

q = Quadrupole(L=0.1, K1=10.)
q

## Quadrupole one particle test

In [ ]:
# Outgoing particle
p_out = track_a_quadrupole_torch(p_in, q)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']])
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

## Quadrupole Jacobian test

In [ ]:
# autodiff Jacobian matrix
f_quadrupole = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), q)[:6]

J = jacobian(f_quadrupole, diff_coords)
mat_py = torch.vstack(J)
mat_py

In [ ]:
# Bmad formula Jacobian
quad_tao = tao.matrix(0,1)
mat_tao = torch.tensor(quad_tao['mat6'])
mat_tao

In [ ]:
# close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-14)

## Quadrupole offset test

In [ ]:
# Create quad with offsetss
q_off = track_quadrupole(L = 0.1,
                         K1 = 10.,
                         X_OFFSET = 1e-3,
                         Y_OFFSET = -2e-3)
q_off

In [ ]:
# Outgoing particle no offset
p_out = track_a_quadrupole_torch(p_in, q)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Outgoing particle offset
p_out = track_a_quadrupole_torch(p_in, q_off)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad_offset.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']])
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

## Quadrupole tilt test (transverse rotation)

In [ ]:
# Create quadrupole with tilt
q_tilt = Quadrupole(L = 0.1,
                    K1 = 10.,
                    TILT = 0.3)
q_tilt

In [ ]:
# Outgoing particle no tilt
p_out = track_element(p_in, q)
x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Outgoing particle tilt
p_out = track_a_quadrupole_torch(p_in, q_tilt)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad_tilt.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']])
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

# Crab_Cavity tests

In [ ]:
# Create cavity
from bmadx import CrabCavity

cav = CrabCavity(L=0.2,
                 VOLTAGE=1e4,
                 PHI0=0.5,
                 RF_FREQUENCY=1e9)
cav

## Crab_cavity one particle test

In [ ]:
# Outgoing particle
p_out = track_element(p_in, cav)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_crab_cavity.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']])
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

## Crab_cavity Jacobian test

In [ ]:
# autodiff Jacobian matrix
f_crab_cavity = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), cav)[:6]

J = jacobian(f_crab_cavity, diff_coords)
mat_py = torch.vstack(J)
mat_py

In [ ]:
# Bmad formula Jacobian
cav_tao = tao.matrix(0,1)
mat_tao = torch.tensor(cav_tao['mat6'])
mat_tao

In [ ]:
# close to Tao result up to 13th decimal place
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-13)

In [ ]:
# but not up to 14th decimal place
torch.isclose(mat_py, mat_tao, atol=0, rtol=1e-14)

## Crab_cavity offset test

In [ ]:
# Cavity with offsets
cav_off = CrabCavity(L=0.2,
                     VOLTAGE=1e4,
                     PHI0=0.5,
                     RF_FREQUENCY=1e9,
                     X_OFFSET=1e-3,
                     Y_OFFSET=-2e-3)
cav_off

In [ ]:
# Outgoing particle no offset
p_out = track_element(p_in, cav)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Outgoing particle offset
p_out = track_element(p_in, cav_off)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_crab_cavity_offset.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']])
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

## Crab_cavity tilt test

In [ ]:
# Cavity with tilts
cav_tilt = CrabCavity(L=0.2,
                      VOLTAGE=1e4,
                      PHI0=0.5,
                      RF_FREQUENCY=1e9,
                      X_OFFSET=1e-3,
                      Y_OFFSET=-2e-3
                      TILT=0.3)
cav_tilt

In [ ]:
# Outgoing particle no offset
p_out = track_element(p_in, cav)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Outgoing particle offset and tilt
p_out = track_element(p_in, cav_tilt)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_crab_cavity_tilt.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']])
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

In [ ]:
f_crab_cavity = lambda x: track_element(Particle(*x,s=p_in.s, p0c=p_in.s, mc2=p_in.mc2), cav_tilt)[:6]

J = jacobian(f_crab_cavity, diff_coords)
mat_py = torch.vstack(J)
mat_py

In [ ]:
cav_tao = tao.matrix(0,1)
mat_tao = torch.tensor(cav_tao['mat6'])
mat_tao

In [ ]:
torch.isclose(mat_py, mat_tao, atol=0, rtol=1.0e-10)

In [ ]:
# Something is wrong with the Jacobian matrix of a tilted cavity. Currently debuging.

# RF_Cavity tests

In [ ]:
# Create cavity
from bmadx import RFCavity

cav = RFCavity(L=0.2,
               VOLTAGE=1e4,
               PHI0=0.5,
               RF_FREQUENCY=1e9)
cav

## RF_Cavity one particle test

In [ ]:
# Outgoing particle
p_out = track_element(p_in, cav)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_rf_cavity.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']])
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

## RF_Cavity Jacobian test

In [ ]:
# autodiff Jacobian matrix
f_rf_cavity = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), cav)[:6]

J = jacobian(f_rf_cavity, diff_coords)
mat_py = torch.vstack(J)
mat_py

In [ ]:
# Bmad formula Jacobian
quad_tao = tao.matrix(0,1)
mat_tao = torch.tensor(quad_tao['mat6'])
mat_tao

In [ ]:
# close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-13)

## RF_Cavity offset test

In [ ]:
# RF cav with offsets
cav_off = RFCavity(L=0.2,
                   VOLTAGE = 1e3,
                   PHI0 = 0.5,
                   RF_FREQUENCY = 1e9,
                   X_OFFSET = 1e-3,
                   Y_OFFSET = -2e-3)
cav_off

In [ ]:
# Outgoing particle no offset
p_out = track_a_rf_cavity_torch(p_in, cav)

x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Outgoing particle offset
p_out = track_a_rf_cavity_torch(p_in, cav_off)
x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_rf_cavity_offset.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']])
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

## RF_Cavity tilt test

In [ ]:
# RF cav with tilt
cav_tilt = RFCavity(L=0.1,
                    VOLTAGE=1e3,
                    PHI0=0.5,
                    RF_FREQUENCY=1e9,
                    X_OFFSET=1e-3,
                    Y_OFFSET=-2e-3,
                    TILT=0.3)
cav_tilt

In [ ]:
# Outgoing particle no offset
p_out = track_a_rf_cavity_torch(p_in, cav)
x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Outgoing particle offset and tilt
p_out = track_a_rf_cavity_torch(p_in, cav_tilt)
x_py = torch.hstack(p_out[:6])
x_py

In [ ]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_rf_cavity_tilt.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']])
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)